Ch3 p79

Make a table for f(x) = exp(-x)\*x^2 for x = 0.2, 0.3, 0.6, 0.9, and 1.0. Construct quadratic interpolating polynomials using three successive points beginning at x = 0.2, x = 0.3, and x = 0.6. What are the errors if we use each of these to estimate f(0.5)? Compare the bounds for the errors.

In [29]:
import numpy as np
from naf.incf import dd_tb, dd_poly
from sympy import symbols, diff, exp
from tabulate import tabulate
import math

In [73]:
def fp(x):
    return np.exp(-x)*x**2

# def fp(x):
#     return x**2*np.exp(-x/2)

x = np.array([0.2,0.3,0.6,0.9,1.0])
#x = np.array([1.1, 2., 3.5, 5., 7.1])
y = fp(x)
pts = np.column_stack((x,y))

tb = dd_tb(pts)
data = np.column_stack((x,y,tb))
print(tabulate(data))

---  ---------  ---------  --------  -----------  ---------  -------
0.2  0.0327492  0.0327492  0.339244   0.242711    -0.339979  0.17732
0.3  0.0666736  0.0666736  0.436328   0.00472603  -0.198123  0
0.6  0.197572   0.197572   0.439164  -0.13396      0         0
0.9  0.329321   0.329321   0.38558    0            0         0
1    0.367879   0.367879   0          0            0         0
---  ---------  ---------  --------  -----------  ---------  -------


In [93]:
xi = 0.5
# xi = 1.75

v1 = dd_poly(tb[0], x[0:3], xi, 2)
v2 = dd_poly(tb[1], x[1:4], xi, 2)
v3 = dd_poly(tb[2], x[2:5], xi, 2)

fp_val = fp(xi)
vals = np.array([v1, v2, v3])
err = fp_val - vals

data = np.column_stack((vals, err))
print(fp_val)
print(tabulate(data))

0.15163266492815836
--------  -----------
0.149085   0.00254755
0.153845  -0.00221215
0.148297   0.00333528
--------  -----------


In [94]:
z = symbols('z')
eq = exp(-z)*z**2
deq3 = diff(eq, z, 3)
print(deq3)

(-z**2 + 6*z - 6)*exp(-z)


In [95]:
def df3(x):
    return (-x**2+6*x-6)*np.exp(-x)

# def df3(x):
#     return (-x**2/8 + 3*x/2 - 3)*np.exp(-x/2)

def error_bounds(xr, x, n):
    if xr.shape[0] != n+1:
        raise ValueError("xr length must equal n")
        
    val = 1.0
    for xi in xr:
        val = (x-xi)*val
    
    dy3 = df3(np.append(xr,x))
    print(dy3)
    fmax = dy3[np.argmax(np.abs(dy3))]
    fmin = dy3[np.argmin(np.abs(dy3))]
        
    eb_max = val*fmax/math.factorial(n+1)
    eb_min = val*fmin/math.factorial(n+1)
    
    return np.array([fmax, fmin, eb_max, eb_min])

eb_max = np.empty(3)
eb_min = np.empty(3)
fmax = np.empty(3)
fmin = np.empty(3)
for i in range(3):
    print(x[i:i+3])
    eb = error_bounds(x[i:i+3], xi, 2)
    fmax[i] = eb[0]
    fmin[i] = eb[1]
    eb_max[i] = eb[2]
    eb_min[i] = eb[3]

headers = ['interpolate value', 'actual error', 'fmax', 'fmin', 'ueb', 'leb']
data = np.column_stack((vals, err, fmax, fmin, eb_max, eb_min))
print(tabulate(data, headers))

[0.2 0.3 0.6]
[-3.96265684 -3.17811017 -1.51472012 -1.97122464]
[0.3 0.6 0.9]
[-3.17811017 -1.51472012 -0.57326322 -1.97122464]
[0.6 0.9 1. ]
[-1.51472012 -0.57326322 -0.36787944 -1.97122464]
  interpolate value    actual error      fmax       fmin          ueb           leb
-------------------  --------------  --------  ---------  -----------  ------------
           0.149085      0.00254755  -3.96266  -1.51472    0.00396266   0.00151472
           0.153845     -0.00221215  -3.17811  -0.573263  -0.00423748  -0.000764351
           0.148297      0.00333528  -1.97122  -0.367879   0.00657075   0.00122626


Goodness that took a while. It's so persnickety. You really have to pay attention to not only the math but also make sure you are passing the correct input to the functions. 